In [3]:
import json
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df_wind = pd.read_csv("us_wind.csv", sep = ',')
df_wind

,case_id,faa_ors,faa_asn,usgs_pr_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,...,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat
0,3073429,missing,missing,4960,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,1/1/2012,NAIP,-118.360725,35.083778
1,3071522,missing,missing,4997,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,1/1/2012,NAIP,-118.361168,35.081512
2,3073425,missing,missing,4957,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,1/1/2012,NAIP,-118.360420,35.084709
3,3071569,missing,missing,5023,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,7/31/2016,Digital Globe,-118.364029,35.079418
4,3005252,missing,missing,5768,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,11/23/2017,Digital Globe,-118.354286,35.085594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58180,3040109,06-037711,2011-WTW-12036-OE,-9999,CA,Yolo County,6113,unknown Yolo County,2014,1,...,-9999.0,-9999.0,-9999.0,28.7,1,3,6/18/2017,Digital Globe,-121.861214,38.580673
58181,3040944,48-024978,2011-WTW-352-OE,41364,TX,Young County,48503,unknown Young County 1,2011,1,...,-9999.0,-9999.0,-9999.0,29.0,1,3,missing,Digital Globe,-98.551094,33.093292
58182,3041125,48-025943,2011-WTW-4884-OE,41365,TX,Young County,48503,unknown Young County 2,2011,1,...,-9999.0,-9999.0,-9999.0,34.1,1,3,missing,Digital Globe,-98.617889,33.094593
58183,3055917,08-072245,2015-WTW-9994-OE,-9999,CO,Yuma County,8125,unknown Yuma County,2016,2,...,-9999.0,-9999.0,-9999.0,48.8,1,3,5/17/2017,Digital Globe,-102.717171,40.035072


In [5]:
df_wind = df_wind.drop( ["t_hh","t_rd","t_rsa", "t_conf_atr", "t_conf_loc","t_fips", "t_img_date", "t_img_srce", "faa_ors","faa_asn", "usgs_pr_id"], axis=1 )

In [6]:
df_wind

,case_id,t_state,t_county,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
0,3073429,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-9999.0,-118.360725,35.083778
1,3071522,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-9999.0,-118.361168,35.081512
2,3073425,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-9999.0,-118.360420,35.084709
3,3071569,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-9999.0,-118.364029,35.079418
4,3005252,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-9999.0,-118.354286,35.085594
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58180,3040109,CA,Yolo County,unknown Yolo County,2014,1,-9999.00,missing,missing,-9999,28.7,-121.861214,38.580673
58181,3040944,TX,Young County,unknown Young County 1,2011,1,-9999.00,missing,missing,-9999,29.0,-98.551094,33.093292
58182,3041125,TX,Young County,unknown Young County 2,2011,1,-9999.00,missing,missing,-9999,34.1,-98.617889,33.094593
58183,3055917,CO,Yuma County,unknown Yuma County,2016,2,-9999.00,missing,missing,-9999,48.8,-102.717171,40.035072


In [7]:
# for index, row in df_wind.iterrows():
#     if(row["t_ttlh"] == -9999.0):
#         print("true")
#         row.at["t_ttlh"] = -1
df_wind['t_ttlh'] = df_wind['t_ttlh'].replace(-9999.0,-1)

In [8]:
df_wind

,case_id,t_state,t_county,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
0,3073429,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.360725,35.083778
1,3071522,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.361168,35.081512
2,3073425,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.360420,35.084709
3,3071569,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.364029,35.079418
4,3005252,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.354286,35.085594
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58180,3040109,CA,Yolo County,unknown Yolo County,2014,1,-9999.00,missing,missing,-9999,28.7,-121.861214,38.580673
58181,3040944,TX,Young County,unknown Young County 1,2011,1,-9999.00,missing,missing,-9999,29.0,-98.551094,33.093292
58182,3041125,TX,Young County,unknown Young County 2,2011,1,-9999.00,missing,missing,-9999,34.1,-98.617889,33.094593
58183,3055917,CO,Yuma County,unknown Yuma County,2016,2,-9999.00,missing,missing,-9999,48.8,-102.717171,40.035072


In [9]:
for index, row in df_wind.iterrows():
    print(row['p_name'])

251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
251 Wind
2

Barton
Barton
Barton
Barton
Barton
Barton
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Barton Chapel
Bayonne Wind Energy Project
Bear Creek
Bear Creek
Bear Creek
Bear Creek
Bear Creek
Bear Creek
Bear Creek
Bear Creek
Be

Block Island
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Bloom
Blue Breezes I & II
Blue Breezes I & II
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Canyon I
Blue Ca

Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buena Vista
Buffalo Bear
Buffalo Bear
Buffalo Bear
Buffalo Bear
Buffalo Bear
Buffalo Bear
Buffalo Bear
Buffalo Bear
Buffalo Bear
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
Buffalo Dunes
B

Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Caprock Wind Ranch
Carleton College
Carleton College
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
Carousel
C

...

Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Phase III
Sweetwater Ph

Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin Groves I
Twin G

White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Oak
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
White Tail
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hill
Whitewater Hi

unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown DeKalb County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dewey County
unknown Dickey County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County
unknown Dixon County

unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Resource Area 1
unknown Tehachapi Wind Re

In [10]:
df_wind.to_csv('wind_usa_cleaned.csv', index=False)

In [11]:
df_wind

,case_id,t_state,t_county,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
0,3073429,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.360725,35.083778
1,3071522,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.361168,35.081512
2,3073425,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.360420,35.084709
3,3071569,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.364029,35.079418
4,3005252,CA,Kern County,251 Wind,1987,194,18.43,Vestas,missing,95,-1.0,-118.354286,35.085594
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58180,3040109,CA,Yolo County,unknown Yolo County,2014,1,-9999.00,missing,missing,-9999,28.7,-121.861214,38.580673
58181,3040944,TX,Young County,unknown Young County 1,2011,1,-9999.00,missing,missing,-9999,29.0,-98.551094,33.093292
58182,3041125,TX,Young County,unknown Young County 2,2011,1,-9999.00,missing,missing,-9999,34.1,-98.617889,33.094593
58183,3055917,CO,Yuma County,unknown Yuma County,2016,2,-9999.00,missing,missing,-9999,48.8,-102.717171,40.035072


In [12]:
import random
randomlist = []
for i in range(0,200):
    n = random.randint(0,58184)
    randomlist.append(n)
print(randomlist)

[38301, 28303, 33940, 5648, 24914, 5223, 2966, 32061, 233, 35033, 27313, 40836, 36428, 1488, 43932, 6176, 6868, 37701, 8248, 58097, 4630, 22770, 45529, 51444, 37358, 3668, 1730, 49569, 483, 51443, 41492, 40064, 57399, 39264, 28947, 2622, 15614, 41611, 33798, 44614, 26961, 34854, 20943, 12801, 33121, 11893, 54460, 44935, 18425, 52174, 13118, 21521, 3514, 7472, 50600, 39372, 47144, 2337, 10145, 5842, 31072, 48958, 18900, 45011, 55089, 53252, 34245, 46079, 30544, 31009, 14754, 35774, 46759, 58076, 52717, 56346, 40224, 24509, 43170, 47696, 27566, 38502, 11625, 17736, 17605, 55923, 17359, 31746, 54574, 44096, 29095, 34069, 55523, 43820, 50075, 52455, 55144, 47922, 5095, 52623, 38991, 74, 45469, 6944, 31163, 31105, 47522, 27084, 35383, 12369, 47151, 29317, 20387, 11537, 21640, 7268, 57739, 27997, 43334, 23038, 19448, 15019, 10493, 7444, 40096, 52149, 29959, 12506, 48793, 33038, 45033, 31790, 23041, 30163, 34119, 22142, 8836, 3987, 55006, 19938, 12626, 5380, 31278, 43169, 33555, 20123, 38596,

In [16]:
df_wind_short=df_wind[df_wind.index.isin(randomlist)]
print(df_wind_short)

       case_id t_state         t_county  \
74     3073431      CA      Kern County   
233    3025840      IA     Adair County   
483    3036507      CA      Kern County   
1310   3049897      OH  Paulding County   
1488   3041427      TX    Kinney County   
...        ...     ...              ...   
56346  3010269      CA      Kern County   
57399  3009845      CA      Kern County   
57739  3001091      CA      Kern County   
58076  3008598      CA      Kern County   
58097  3010374      CA      Kern County   

                                       p_name  p_year  p_tnum    p_cap  \
74                                   251 Wind    1987     194    18.43   
233                                     Adair    2008      76   174.80   
483                                    Alta I    2010     100   150.00   
1310              Amazon Wind Farm US Central    2016      48   100.80   
1488                                 Anacacho    2012      55    99.00   
...                                    

In [17]:
df_wind_short.to_csv('wind_usa_cleaned_st.csv', index=False)